In [1]:
!git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd /content/Hierarchical-Localization
!pip install -e .
!pip install --upgrade --quiet plotly

import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm 
from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

/content/Hierarchical-Localization
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/Hierarchical-Localization
     |████████████████████████████████| 12.7 MB 6.1 MB/s 
     |████████████████████████████████| 565 kB 66.7 MB/s 
  Running setup.py develop for hloc
     |████████████████████████████████| 15.2 MB 6.8 MB/s 


# Video to frames

In [2]:
!pip install youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.3 MB/s 


In [ ]:
# # Next, we cut out part of the youtube video
# !ffmpeg -i $(youtube-dl -f 18 --get-url https://www.youtube.com/watch?v=_y3MAH65Cfg&t=115s) \
# -ss 00:02:00 -t 00:02:20 -c:v copy -c:a copy\
# data.mp4

In [3]:
!ffmpeg -ss 00:00:41 -i $(youtube-dl -f 136 --get-url https://www.youtube.com/watch?v=Mq8LFUyvBrM&t=41s) -t 10 -c:v libx264 -c:a copy data.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
!mkdir -p bottom
!ffmpeg -i data.mp4 -r 1 bottom/img%04d.jpg -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'data.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
  Duration: 00:00:10.01, start: 0.000000, bitrate: 1225 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1280x720 [SAR 1:1 DAR 16:9], 1222 kb/s, 29.97 fps, 29.97 tbr, 30k tbn, 59.94 tbc (default)
    Metadata:
      handler_name    : VideoHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> mjpeg (native))
Press [q] to stop, [?] for help
[swscaler @ 0x5646423dc000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'bottom/img%04d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
    Stream #0:0(und): Video: mjpeg, yuvj420p(pc), 1280x720 [SAR 1:1 DAR 16:9], q=2-31, 200 kb/s, 1 fps, 1 tbn, 1 tbc (default)
    Metadata:
      h

# Setup
Here we define some output paths.

In [ ]:
images = Path('bottom')
outputs = Path('outputs/demo/')
!rm -rf $outputs
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

# 3D mapping
First we list the images used for mapping. 

In [ ]:
references = [str(p.relative_to(images)) for p in (images).iterdir()]

Then we extract features and match them across image pairs. Since we deal with few images, we simply match all pairs exhaustively.


In [ ]:
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

[2022/08/31 17:02:34 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}


Loaded SuperPoint model


  0%|          | 0/12 [00:00<?, ?it/s]

[2022/08/31 17:02:35 hloc INFO] Finished exporting features.
[2022/08/31 17:02:35 hloc INFO] Found 66 pairs.
[2022/08/31 17:02:35 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


  0%|          | 0/66 [00:00<?, ?it/s]

[2022/08/31 17:02:47 hloc INFO] Finished exporting matches.


The we run incremental Structure-from-Motion and display the reconstructed 3D model.

In [ ]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=references)
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping")
fig.show()

[2022/08/31 17:03:24 hloc INFO] Creating an empty database...
[2022/08/31 17:03:24 hloc INFO] Importing images into the database...
[2022/08/31 17:03:24 hloc INFO] Importing features into the database...


  0%|          | 0/12 [00:00<?, ?it/s]

[2022/08/31 17:03:24 hloc INFO] Importing matches into the database...


  0%|          | 0/66 [00:00<?, ?it/s]

[2022/08/31 17:03:24 hloc INFO] Performing geometric verification of the matches...
[2022/08/31 17:03:25 hloc INFO] Running 3D reconstruction...


In [ ]:
from hloc.utils.read_write_model import read_model
cameras, imges, points3D = read_model('/content/Hierarchical-Localization/outputs/demo/sfm', ext='.bin')

We also visualize which keypoints were triangulated into the 3D model.

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='visibility', n=2)

Saving our point cloud to a file

In [ ]:
from hloc.utils import read_write_model
model.export_PLY("crabs.ply")

# Open3D Poisson Surface

Вопрос: https://github.com/isl-org/Open3D/issues/5439

In [ ]:
!pip install surface_reconstruction

In [ ]:
!pip3 install open3d

In [ ]:
import open3d as o3d
o3d.geometry.PointCloud

open3d.cuda.pybind.geometry.PointCloud

In [ ]:
from surface_reconstruction import SurfaceReconstruction
import os

In [ ]:
# Pass a method/library that contains a Poisson algorithm implementation
surface = SurfaceReconstruction(
  method_type='open3d',
  point_cloud_file=os.path.join('/content','Hierarchical-Localization', 'crabs.ply'),
  output_file=os.path.join('/content','terrain_mesh2.ply'))


surface.poisson_mesh()